In [2]:
import pandas as pd
from dask import dataframe as dd

In [3]:
%%writefile testutility.py
import logging
import os
import subprocess
import pandas as pd
import datetime
import gc
import re
import yaml

###function to read the yaml file
def read_conf(filepath):
    with open(filepath , "r") as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)

###function to validate the column names
def field_validation(df,conf):
    ###convert all column names in the dataframe to lower case    
    df.columns = [x.lower() for x in df.columns]
    ### convert all column names in the config file to lower case
    conf_columns =[x.lower() for x in conf["columns"]]
    
    ### Remove all white spaces and replace them with underscores
    for col in df.columns:
        if " " in col:
            df = df.rename(columns={col:col.replace(" ","_")})
    ### removing all the underscore for comparison purposes
    df.columns = [x.strip("_") for x in df.columns]
    conf_columns = [x.strip("_") for x in conf["columns"]]
    
    ###comparing the two column names
    if list(df.columns) == list(conf_columns):
        return 1
    else:
        missing_columns =list(set(df.columns).difference(conf_columns))
        print("The following columns are not in the yaml file", missing_columns)
        missing_yaml_file =list(set(conf_columns).difference(df.columns))
        print("The following columns are not in the uploaded file", missing_yaml_file)
        return 0
        
        
                      


Overwriting testutility.py


In [4]:
%%writefile config.yaml
filetype: csv
dataset_name: agg_match_stats_2
inbound_delimeter: ","
oubound_delimeter: "|"
skip_leading_rows: 1
columns : ['date', 'game_size', 'match_id', 'match_mode', 'party_size',
       'player_assists', 'player_dbno', 'player_dist_ride', 'player_dist_walk',
       'player_dmg', 'player_kills', 'player_name', 'player_survive_time',
       'team_id', 'team_placement']

Overwriting config.yaml


In [5]:
import testutility as util

In [6]:
conf_file = util.read_conf("config.yaml")

In [7]:
conf_file

{'filetype': 'csv',
 'dataset_name': 'agg_match_stats_2',
 'inbound_delimeter': ',',
 'oubound_delimeter': '|',
 'skip_leading_rows': 1,
 'columns': ['date',
  'game_size',
  'match_id',
  'match_mode',
  'party_size',
  'player_assists',
  'player_dbno',
  'player_dist_ride',
  'player_dist_walk',
  'player_dmg',
  'player_kills',
  'player_name',
  'player_survive_time',
  'team_id',
  'team_placement']}

In [8]:
file_type = conf_file["filetype"]
file_path = "./" + conf_file["dataset_name"] + f".{file_type}"
data = dd.read_csv(file_path)

In [9]:
util.field_validation(data,conf_file)

1

In [10]:
if util.field_validation(data,conf_file)==0:
    print("Column validation failed")
else:
    print("Success.All columns in place!!!")

Success.All columns in place!!!


In [11]:
data.to_csv("./compressed_file.csv.gz" , sep ="|" , compression ="gzip")

C:\Users\User\anaconda3\lib\site-packages\dask\core.py:118: DtypeWarning: Columns (9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  args2 = [_execute_task(a, cache) for a in args]


ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+----------------+---------+----------+
| Column         | Found   | Expected |
+----------------+---------+----------+
| player_dmg     | object  | int64    |
| player_kills   | object  | int64    |
| team_placement | float64 | int64    |
+----------------+---------+----------+

The following columns also raised exceptions on conversion:

- player_dmg
  ValueError("invalid literal for int() with base 10: '47\\x0c0'")
- player_kills
  ValueError("invalid literal for int() with base 10: 'ooodesxy'")

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'player_dmg': 'object',
       'player_kills': 'object',
       'team_placement': 'float64'}

to the call to `read_csv`/`read_table`.